In [1]:
import pandas as pd
from utils import preprocess_text
import random

In [2]:
data_dir = "/Users/masang/Desktop/side-projects/ko2en/data/aihub_news_data.csv"

df = pd.read_csv(data_dir)
df.sample(5)

,원문,번역문
85298,지난해에 이어 두 번째로 실시한 이번 행사에는 에어부산 캐빈승무원들과 대구지점 봉사...,"The event, the second of its kind since last y..."
49203,그나마 정부가 가격을 통제하는 전기·수도·가스 가격은 3.3% 하락하면서 지난해 1...,"At least, prices of electricity, water and gas..."
79217,나는 박길종과 김윤하가 작업을 시작한 시기부터 줄곧 보아왔는데 처음엔 그들이 제작하...,I've seen Park Gil-jong and Kim Yoon-ha since ...
591550,"국토부 부동산평가과 관계자는 “종부세는 1주택보다는 다주택자에게 많이 부과되고, 셈...","""The comprehensive data will not be available ..."
289901,지구에서 밤하늘의 달과 별을 바라보며 꿈을 꾸던 사람들이 있었다.,There were people on the Earth who dreamed whi...


In [3]:
print(len(df))

801387


In [17]:
idx = random.randint(0, len(df) - 1)

processed_ko_text = preprocess_text(df['원문'][idx], lang='ko')
processed_en_text = preprocess_text(df['번역문'][idx], lang='en')

print("Original Korean Text:", df['원문'][idx])
print("Processed Korean Text:", processed_ko_text)
print("Original English Text:", df['번역문'][idx])
print("Processed English Text:", processed_en_text)

Original Korean Text: 김 참모총장도 “장병들이 청춘책방에서 미래를 잘 준비해 전역 후에는 국가 발전을 선도하는 혁신적 인재로 클 수 있길 기대한다”고 말했다.
Processed Korean Text: 김 참모총장도 장병들이 청춘책방에서 미래를 잘 준비해 전역 후에는 국가 발전을 선도하는 혁신적 인재로 클 수 있길 기대한다 고 말했다.
Original English Text: "We hope that the generals will prepare well for the future in the youth bookstore and grow up to be innovative people who lead the nation's development after they leave the military," the chief of staff Kim said.
Processed English Text: We hope that the generals will prepare well for the future in the youth bookstore and grow up to be innovative people who lead the nation's development after they leave the military the chief of staff Kim said.
